In [1]:
import numpy as np
from data import get_data
from model import *
from model_constraints import *

Get Data

In [2]:
instance = 6
np.random.seed(instance)
data = get_data(f"output/instance{instance}/instance{instance}.txt")

Create the model

In [3]:
model = create_model()

Create the Decision Variables

In [4]:
employee_dv = create_employee_dv(model, "assigned", data["STAFF"], data["DAYS"], data["SHIFTS"])
above_dv = create_above_coverage_dv(model, len(data["STAFF"]), "above", data["DAYS"], data["SHIFTS"], data["SCENARIOS"])
below_dv = create_below_coverage_dv(model, len(data["STAFF"]), "below", data["DAYS"], data["SHIFTS"], data["SCENARIOS"])
weekend_dv = create_weekend_dv(model, "weekend", data["STAFF"], data["WEEKENDS"])
add_dv = create_add_dv(model, len(data["STAFF"]) // 2, "above", data["DAYS"], data["SHIFTS"], data["SCENARIOS"])
subtract_dv = create_subtract_dv(model, len(data["STAFF"]) // 2, "below", data["DAYS"], data["SHIFTS"], data["SCENARIOS"])

Apply Constraints

In [5]:
one_shift_per_day(model, data, employee_dv)
shift_rotation(model, data, employee_dv)
max_shifts(model, data, employee_dv)

max_work_time(model, data, employee_dv)
min_work_time(model, data, employee_dv)

max_consecutive_shifts(model, data, employee_dv)
min_consecutive_shifts(model, data, employee_dv)
min_consecutive_shifts_off(model, data, employee_dv)

max_weekends(model, data, employee_dv, weekend_dv)

days_off(model, data, employee_dv)

coverage2(model, data, employee_dv, above_dv, below_dv, add_dv, subtract_dv, data["SCENARIOS"])

add_staff(model, data, add_dv, below_dv, data["SCENARIOS"])
remove_staff(model, data, subtract_dv, above_dv, data["SCENARIOS"])

SOLVE

In [6]:
# All constraints together takes 1.5 hours to find a feasible solution
import time

start = time.time()

model.Minimize(
        #off requests not applied
        sum(data["SECTION_SHIFT_OFF_REQUESTS"].get((s, d - 1, t), 0) * employee_dv[(s, d, t)]
            for s in data["STAFF"]
            for d in data["DAYS"]
            for t in data["SHIFTS"])
        +
        #on requests not applied
        sum(data["SECTION_SHIFT_ON_REQUESTS"].get((s, d - 1, t), 0) * (1 - employee_dv[(s, d, t)])
            for s in data["STAFF"]
            for d in data["DAYS"]
            for t in data["SHIFTS"])
        +
        sum([
        #overstaffing penalty
        (1 / len(data["SCENARIOS"])) * sum(above_dv[d, t, k] * [item[4] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0]
            for d in data["DAYS"]
            for t in data["SHIFTS"]
            for k in data["SCENARIOS"])
        + 
        #understaffing penalty
        (1 / len(data["SCENARIOS"])) * sum(below_dv[d, t, k] * [item[3] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0]
            for d in data["DAYS"]
            for t in data["SHIFTS"]
            for k in data["SCENARIOS"])
        + 
        #readd
        (1 / len(data["SCENARIOS"])) * sum(add_dv[d, t, k] * ([item[3] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0] * 0.75)
            for d in data["DAYS"]
            for t in data["SHIFTS"]
            for k in data["SCENARIOS"])
        + 
        #remove
        (1 / len(data["SCENARIOS"])) * sum(subtract_dv[d, t, k] * ([item[4] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0] * 0.75)
            for d in data["DAYS"]
            for t in data["SHIFTS"]
            for k in data["SCENARIOS"])
        ]
        )
)

solver = create_solver(1200)
status = solver.Solve(model)

print(solver.StatusName(status))
import collections
results = collections.defaultdict(list)
for d in data["DAYS"]:
    results[d] = []

with open(f"output/instance{instance}/instance{instance}s_fa.txt", 'w') as file:
    file.write(f"{status}\n")
    file.write('Maximum of objective function: %i' % solver.ObjectiveValue() + '\n\n')
    print()
    for d in data["DAYS"]:
        file.write(f'Day: {d}\n')
        for s in data["SHIFTS"]:
            for m in data["STAFF"]:
                if solver.Value(employee_dv[(m, d, s)]) == 1:
                    results[d].append(m)
                    file.write(f"{m} works shift {s}\n")
        file.write("\n")

        end = time.time()
    print(f"time elapsed: {end - start}")

FEASIBLE

time elapsed: 1199.4891057014465
